## Keras EfficientNet example
[Link to tutorial](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

Content
1. [AWS BucketSetup](#1.-AWS-Bucket-setup)

In [2]:
%matplotlib inline
import matplotlib as plt
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['figure.dpi'] = 72

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

In [ ]:
%pip install --upgrade sagemaker
%pip install tensorflow_datasets

In [ ]:
import os
import sys
import json
import glob
import boto3
import pandas as pd
import matplotlib.pyplot as plt

import sagemaker
from sagemaker import get_execution_role, image_uris, model_uris, script_uris, hyperparameters
from sagemaker.s3 import S3Downloader
from sagemaker.utils import name_from_base
from sagemaker.session import Session
from sagemaker.estimator import Estimator
from sagemaker.analytics import TrainingJobAnalytics
from sagemaker import exceptions
from sagemaker.tuner import (
    HyperparameterTuner,
    ContinuousParameter,
    IntegerParameter,
    CategoricalParameter,
)
from botocore.exceptions import ClientError
import tensorflow_datasets as tfds

## 1. AWS Bucket setup

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:
# Download data for preprocessing
solution_bucket = "sagemaker-solutions-prod"
solution_name = "sagemaker-defect-detection/1.4.0"

original_bucket = f"s3://{solution_bucket}-{region}/{solution_name}"
original_data_prefix = "data/NEU-DET.zip"
original_data = f"{original_bucket}/{original_data_prefix}"
print("original data: ")
S3Downloader.list(original_data)

## Write the output of the model predictions in key with the datetime in it.
I couldn't manage to save the input data in S3. So we load it in again each run.
Probably should have loaded it ecs, unzipped it and copied the output to S3.

In [ ]:
topic_prefix = 'keras-examples'
output_files_prefix = name_from_base('efficientnet-classification')


s3_output_location = f"s3://{bucket}/{topic_prefix}/{output_files_prefix}/output"
print(s3_output_location)

In [ ]:
batch_size = 64

dataset_name = "stanford_dogs"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

In [ ]:
## Keras example
# input shape of (224, 224, 3)
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

In [ ]:
model = EfficientNetB0(include_top=False, weights='imagenet')

In [ ]:
model = EfficientNetB0(weights='imagenet', drop_connect_rate=0.4)

## Stanford dogs

In [ ]:
IMG_SIZE = 224

In [ ]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

In [ ]:
import matplotlib.pyplot as plt


def format_label(label):
    # this is a method on tfds.features.ClassLabel https://www.tensorflow.org/datasets/api_docs/python/tfds/features/ClassLabel#int2str
    string_label = label_info.int2str(label)
    return string_label.split("-")[1]


label_info = ds_info.features["label"]
for i, (image, label) in enumerate(ds_train.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title("{}".format(format_label(label)))
    plt.axis("off")

In [ ]:
# data augmentation step
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
for image, label in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        # I thnk this is done to add an outer batch dimension.
        # Sometimes tf.nn.conv2d (or the tf network being used requires certain dimensionality)
        aug_img = img_augmentation(tf.expand_dims(image, axis=0))
        plt.imshow(aug_img[0].numpy().astype("uint8"))
        plt.title("{}".format(format_label(label)))
        plt.axis("off")

In [ ]:
# One-hot / categorical encoding
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

with strategy.scope():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.summary()

epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)